In [51]:
import os

In [52]:
os.chdir('./chicken-disease-classification/')

FileNotFoundError: [WinError 2] The system cannot find the file specified: './chicken-disease-classification/'

In [53]:
%pwd

'd:\\Vedaant\\MachineLearningTutorial\\Chicken-Disease-Classification\\chicken-disease-classification'

In [61]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [62]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [63]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [64]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [65]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exits of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts a zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [67]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-19 18:17:00,008: INFO: common: Successfully loaded YAML file: config\config.yaml]
[2025-01-19 18:17:00,012: INFO: common: Successfully loaded YAML file: params.yaml]
[2025-01-19 18:17:00,014: INFO: common: Created directory: artifacts]
[2025-01-19 18:17:00,016: INFO: common: Created directory: artifacts/data_ingestion]
[2025-01-19 18:17:00,018: INFO: 1890465784: File already exits of size: 11345 KB]
